# Modeling with Support Vector Regression
- The support vector machine (SVM) is a data-classification algorithm that assigns new data elements to one of the labeled categories, it assumes that the data in question contains some possible target values. In Machine Learning or Data Science are quite familiar with the term Support Vector Machine, but Support Vector Regression (SVR) is a bit different from SVM. As the name suggests the SVR is a regression algorithm, so we can use SVR for working with continuous Values instead of Classification which is SVM. The basic idea of SVR is to find a function f(y) that has most ε deviation from the actually obtained target for the training data yi, and at the same time is as flat as possible. In other words, **we do not care about the errors as long as they are less than ε**. **This property determines the SVR to be less sensitive to outliers than the quadratic loss function.**

## 1. Preprocessing
- Label Encoding
- Log nomalization
- Standard scaling